In [1]:
import numpy as np
import pandas as pd
import docx2txt
import os
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
train_df = pd.read_csv('train.csv')
train_df.head()

,CandidateID,Designation,Have you Completed your Graduation ?,Highest Educational Qualification,Total no of years Experience [before joining Piramal],Previous Industry worked with [before joining Piramal],Name of your Previous Organization / Company,How many Organization that you have worked before joining Piramal Finance ?,Average Incentive [per month] earned in your pervious company ?,How did you come to know about the role at Piramal Finance ?,...,What was the average ticket size handled at your end in previous role ?,How many members are there in your family ?,How many are earning family members ? [Other then yourself]2,How many members are dependent on you ?,Department,DOJ,Location Code,Residential Pincode,Branch Pincode,Performance
0,EMP0669,DST,Full Time,Graduate,5.0,NBFC,ADITYA BIRLA CAPITAL FINANCE,03-May,Above 10K,Others,...,INR 5L - INR 15L,1 - 2 members,1.0,1,Mass Affluent Housing,04-05-23,6333,500036,500016.0,0
1,EMP0273,DST,Full Time,Graduate,1.5,Banking,IDFC FIRST BANK LIMITED,01-Feb,7K-10K,Advertisement / Job Portal,...,INR 50K - INR 2L,3 - 4 members,2.0,4,Sales,07-11-23,6048,621210,620001.0,1
2,EMP0300,DST,Full Time,Under Graduate,4.5,Banking,Onerufe Resource private limited,03-May,Less than 3K,Referral,...,INR 15L and above,3 - 4 members,1.0,2,Affordable Housing,13-02-23,6329,627808,627811.0,1
3,EMP0100,DST,Full Time,Post Graduate,4.5,Banking,Kotak Mahindra Bank Ltd,01-Feb,Above 10K,Consultant / Partners,...,INR 15L and above,3 - 4 members,2.0,3,Affordable Housing,14-02-22,6263,221003,221010.0,1
4,EMP0319,DST,Full Time,Post Graduate,1.0,others,R.G Enterprises,01-Feb,Nil,others,...,Fresher,3 - 4 members,1.0,3,Affordable Housing,20-04-23,6059,636004,636004.0,0


In [3]:
number_columns = list(train_df.describe().columns)
number_columns

['Total no of years Experience [before joining Piramal]',
 'How many are earning family members ? [Other then yourself]2',
 'How many members are dependent on you ?',
 'Location Code',
 'Residential Pincode',
 'Branch Pincode',
 'Performance']

In [4]:
train_df.describe()

,Total no of years Experience [before joining Piramal],How many are earning family members ? [Other then yourself]2,How many members are dependent on you ?,Location Code,Residential Pincode,Branch Pincode,Performance
count,6.710000e+02,563.000000,745.000000,745.000000,745.000000,743.000000,745.000000
mean,4.625170e+04,6282.300178,2.573154,5916.322148,443646.022819,441313.527591,0.395973
std,1.197978e+06,48964.725585,1.447218,1005.556894,188146.764093,188283.541525,0.489387
min,0.000000e+00,0.000000,0.000000,2050.000000,101213.000000,110019.000000,0.000000
25%,2.000000e+00,1.000000,2.000000,6040.000000,274509.000000,273001.000000,0.000000
50%,4.000000e+00,1.000000,2.000000,6139.000000,500008.000000,500014.000000,0.000000
75%,6.000000e+00,2.000000,3.000000,6292.000000,605003.000000,602552.000000,1.000000
max,3.103202e+07,600000.000000,10.000000,6525.000000,847224.000000,842002.000000,1.000000


In [5]:
outliers_dict = {}
for column_name in number_columns:
    iqr = train_df[column_name].quantile(0.75) - train_df[column_name].quantile(0.25)
    outliers = [value for value in train_df[(train_df[column_name] < train_df[column_name].quantile(0.25) - 1.5 * iqr) | (train_df[column_name] > train_df[column_name].quantile(0.95) + 1.5 * iqr)][column_name]]
    outliers_dict[column_name] = outliers
outliers_dict
col1 = 'Total no of years Experience [before joining Piramal]'
col2 = 'How many are earning family members ? [Other then yourself]2'
train_df.loc[train_df[col1] == outliers_dict[col1][1], col1] = 4
for i in range(len(outliers_dict[col2])):
    train_df.loc[train_df[col2] == outliers_dict[col2][i], col2] = 1


<h4>There are no outliers in location code(since location codes that I got as outliers are also valid), residential pincode, branch pincode and performance. Increasing the upper bound I got all the outliers in the remaining columns<br> </h4>

In [6]:
train_df.describe()

,Total no of years Experience [before joining Piramal],How many are earning family members ? [Other then yourself]2,How many members are dependent on you ?,Location Code,Residential Pincode,Branch Pincode,Performance
count,671.000000,563.000000,745.000000,745.000000,745.000000,743.000000,745.000000
mean,4.278571,1.532860,2.573154,5916.322148,443646.022819,441313.527591,0.395973
std,3.230621,1.729039,1.447218,1005.556894,188146.764093,188283.541525,0.489387
min,0.000000,0.000000,0.000000,2050.000000,101213.000000,110019.000000,0.000000
25%,2.000000,1.000000,2.000000,6040.000000,274509.000000,273001.000000,0.000000
50%,4.000000,1.000000,2.000000,6139.000000,500008.000000,500014.000000,0.000000
75%,6.000000,2.000000,3.000000,6292.000000,605003.000000,602552.000000,1.000000
max,20.000000,25.000000,10.000000,6525.000000,847224.000000,842002.000000,1.000000


In [7]:
pd.set_option('display.max_columns', None)
train_df.isnull().sum()

CandidateID                                                                      0
Designation                                                                      0
Have you Completed your Graduation ?                                           101
Highest Educational Qualification                                              101
Total no of years Experience [before joining Piramal]                           74
Previous Industry worked with [before joining Piramal]                           0
Name of your Previous Organization / Company                                   192
How many Organization that you have worked before joining Piramal Finance ?     74
Average Incentive [per month] earned in your pervious company ?                  0
How did you come to know about the role at Piramal Finance ?                     0
Which Products you are selling in your pervious role ?                           6
What was the average ticket size handled at your end in previous role ?          0
How 

In [8]:
train_df[train_df['Have you Completed your Graduation ?'] == 'Full Time']['Have you Completed your Graduation ?'].count()

614

In [14]:
unique = train_df.nunique()
unique

CandidateID                                                                    745
Designation                                                                      1
Have you Completed your Graduation ?                                             3
Highest Educational Qualification                                                5
Total no of years Experience [before joining Piramal]                           81
Previous Industry worked with [before joining Piramal]                           6
Name of your Previous Organization / Company                                   486
How many Organization that you have worked before joining Piramal Finance ?      4
Average Incentive [per month] earned in your pervious company ?                  5
How did you come to know about the role at Piramal Finance ?                     8
Which Products you are selling in your pervious role ?                          58
What was the average ticket size handled at your end in previous role ?          8
How 

In [17]:
col = 'Have you Completed your Graduation ?'
col3 = 'Highest Educational Qualification'
col4 = 'Total no of years Experience [before joining Piramal]'
col5 = 'How many Organization that you have worked before joining Piramal Finance ?'  # remove this column not able to make sense of the data here.
col6 = 'Name of your Previous Organization / Company'   # already included in resume and also there are 487 companies here. Too much noise
col7 = 'How many are earning family members ? [Other then yourself]2'
col8 = 'Branch Pincode'
col9 = 'Residential Pincode'

In [18]:
train_df[col] = train_df[col].fillna('Full Time')
train_df[col3] = train_df[col3].fillna('Graduate')
train_df[col4] = train_df[col4].fillna(0)
# train_df[col5] = train_df[col5].fillna(0)
train_df[col6] = train_df[col6].fillna(0)
train_df[col7] = train_df[col7].fillna(1)
train_df[col8] = train_df[col8].fillna(train_df[col9])

In [19]:
train_df.dtypes

CandidateID                                                                     object
Designation                                                                     object
Have you Completed your Graduation ?                                            object
Highest Educational Qualification                                               object
Total no of years Experience [before joining Piramal]                          float64
Previous Industry worked with [before joining Piramal]                          object
Name of your Previous Organization / Company                                    object
How many Organization that you have worked before joining Piramal Finance ?     object
Average Incentive [per month] earned in your pervious company ?                 object
How did you come to know about the role at Piramal Finance ?                    object
Which Products you are selling in your pervious role ?                          object
What was the average ticket size handled at

In [20]:
train_df.describe()

,Total no of years Experience [before joining Piramal],How many are earning family members ? [Other then yourself]2,How many members are dependent on you ?,Location Code,Residential Pincode,Branch Pincode,Performance
count,745.000000,745.000000,745.000000,745.000000,745.000000,745.000000,745.000000
mean,3.853586,1.402685,2.573154,5916.322148,443646.022819,441231.268456,0.395973
std,3.322461,1.520114,1.447218,1005.556894,188146.764093,188037.802864,0.489387
min,0.000000,0.000000,0.000000,2050.000000,101213.000000,110019.000000,0.000000
25%,1.000000,1.000000,2.000000,6040.000000,274509.000000,273001.000000,0.000000
50%,3.400000,1.000000,2.000000,6139.000000,500008.000000,500014.000000,0.000000
75%,6.000000,2.000000,3.000000,6292.000000,605003.000000,602001.000000,1.000000
max,20.000000,25.000000,10.000000,6525.000000,847224.000000,842002.000000,1.000000


In [5]:
def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt:
        txt = re.sub(r'\s+', ' ', txt)
        txt = re.sub(r'Email id:.*?(?=Languages Known:)', '', txt, flags=re.DOTALL)
        txt = re.sub(r'Contact number:.*?(?=Languages Known:)', '', txt, flags=re.DOTALL)
        txt = re.sub(r'Present Address.*?(?=Permanent Address)', '', txt, flags=re.DOTALL)
        txt = re.sub(r'Permanent Address.*?(?=Languages Known:)', '', txt, flags=re.DOTALL)
        txt.replace(" Evaluation Warning: The document was created with Spire.PDF for Python.", "")
        return txt.replace('\t', ' ').replace('\n', ' ')
    return None

txt = extract_text_from_docx('CV train/EMP0008 Resume.docx')


In [6]:
path = 'CV test/'
file_paths = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

In [7]:
def get_resume_texts(file_paths):
    resume_text = []
    for path in file_paths:
        text = extract_text_from_docx(path)
        resume_text.append(text)
    return resume_text

In [8]:
resume_text = get_resume_texts(file_paths)

In [9]:
import pickle
with open("resume_data.pkl", "wb") as file:
    pickle.dump(resume_text, file)

In [26]:
def get_avg_percentage(txt):
    percentages = re.findall(r"(?:\d+(?:\.\d*)?|\d*\.\d+)\%", txt)
    percentages_float = [float(percentage.replace('%', '')) for percentage in percentages]
    if percentages:
        return round(sum(percentages_float) / len(percentages_float), 2)
    else:
        return 0

In [ ]:
score = [get_avg_percentage(txt) for txt in resume_text]
print('zeros: ', score.count(0), 'total: ', len(score))

In [28]:
def extract_skills_from_resume(text, skills_list):
    skills = []

    for skill in skills_list:
        pattern = r"\b{}\b".format(re.escape(skill))
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            skills.append(skill)

    return skills

In [29]:
columns = list(train_df.columns)
columns

['CandidateID',
 'Designation',
 'Have you Completed your Graduation ?',
 'Highest Educational Qualification',
 'Total no of years Experience [before joining Piramal]',
 'Previous Industry worked with [before joining Piramal]',
 'Name of your Previous Organization / Company',
 'How many Organization that you have worked before joining Piramal Finance ?',
 'Average Incentive [per month] earned in your pervious company ?',
 'How did you come to know about the role at Piramal Finance ?',
 'Which Products you are selling in your pervious role ?',
 'What was the average ticket size handled at your end in previous role ?',
 'How many members are there in your family ?',
 'How many are earning family members ? [Other then yourself]2',
 'How many members are dependent on you ?',
 'Department',
 'DOJ',
 'Location Code',
 'Residential Pincode',
 'Branch Pincode',
 'Performance']

In [30]:
train_df["Performance"].value_counts() 

Performance
0    450
1    295
Name: count, dtype: int64

In [31]:
columns_to_drop = ['Name of your Previous Organization / Company', 'How many Organization that you have worked before joining Piramal Finance ?',
                  'CandidateID', 'Designation', 'DOJ', 'Location Code']
columns_to_one_hot = ['Previous Industry worked with [before joining Piramal]', 'Average Incentive [per month] earned in your pervious company ?',
                     'How did you come to know about the role at Piramal Finance ?', 'Which Products you are selling in your pervious role ?',
                     'What was the average ticket size handled at your end in previous role ?', 'How many members are there in your family ?',
                     'Department']
# concatenate 'Have you Completed your Graduation ?' and 'Highest Educational Qualification' 
# use 'Residential Pincode' and 'Branch Pincode' to create distance column and remove those two.

In [32]:
train_df = train_df.drop(columns=columns_to_drop)
(train_df.columns)

Index(['Have you Completed your Graduation ?',
       'Highest Educational Qualification',
       'Total no of years Experience [before joining Piramal]',
       'Previous Industry worked with [before joining Piramal]',
       'Average Incentive [per month] earned in your pervious company ?',
       'How did you come to know about the role at Piramal Finance ?',
       'Which Products you are selling in your pervious role ?',
       'What was the average ticket size handled at your end in previous role ?',
       'How many members are there in your family ?',
       'How many are earning family members ? [Other then yourself]2',
       'How many members are dependent on you ?', 'Department',
       'Residential Pincode', 'Branch Pincode', 'Performance'],
      dtype='object')

In [104]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)  # Set sparse=False for dense output
encoded_data = encoder.fit_transform(train_df[['Previous Industry worked with [before joining Piramal]']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out())

In [78]:
import torch
from sklearn.feature_extraction.text import CountVectorizer

terms = train_df["How did you come to know about the role at Piramal Finance ?"].tolist()

# Create the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the data
X = vectorizer.fit_transform(terms)

# Convert the sparse matrix to a PyTorch tensor
X_tensor = torch.from_numpy(X.toarray())


# Get the vocabulary
vocab = vectorizer.get_feature_names_out()
vocab

array(['advertisement', 'application', 'campus', 'consultant', 'direct',
       'external', 'job', 'others', 'partners', 'portal', 'referral'],
      dtype=object)

In [33]:
with open("resume_data_new.pkl", "rb") as file:
    resume_text_new = pickle.load(file)

In [34]:
resume_text_new = [txt.replace('\n    Summary:\n  ', '') for txt in resume_text_new]

In [35]:
resume_text_new[0]

'  ASUHU is a highly skilled and experienced sales manager with a strong academic background. He holds a Master of Commerce degree and has completed courses in computer concepts and accounting. He has experience working as a manager-in-charge in a company authorized service station dealing with air brake and power steering equipment. His work experience includes managing customer accounts, taking orders, managing dispatches, and promoting the sales volume. He is also proficient in using accounting software such as Tally and Busy. ASUHU is fluent in Hindi and English and enjoys singing and traveling in his free time.'

In [70]:
resume_txt = pd.DataFrame(resume_text_new, columns=['resume_txt'])
df = pd.concat([train_df, resume_txt], axis=1)
df['education'] = df['Have you Completed your Graduation ?'] + ' ' + df['Highest Educational Qualification']
df = df.drop(['Have you Completed your Graduation ?', 'Highest Educational Qualification', 'Residential Pincode', 'Branch Pincode'], axis=1)

df['products'] = df['Which Products you are selling in your pervious role ?'].str.split(';')
all_products = pd.concat([pd.Series(row) for row in df['products']], ignore_index=True).unique()
len(all_products)

16

In [71]:
columns = list(df.columns)
df.to_csv('new_data.csv', index=False)

In [72]:
df['resume_txt']

0        ASUHU is a highly skilled and experienced sa...
1        Candidate Name: S.Thanga Durai\n    Relevant...
2        Harun is an experienced finance professional...
3       Knowledge of software skills in asset manage...
4        This candidate has over one year of experien...
                             ...                        
740      This candidate has extensive experience in m...
741      B. Ravi XXX is an experienced banking profes...
742      This candidate has over 7 years of experienc...
743      Name: [REDACTED]\n    Education:\n    - Govt...
744      This candidate has over 20 years of experien...
Name: resume_txt, Length: 745, dtype: object

<h2>Don't run these</h2>

In [ ]:
def create_pipe(clf):

    column_trans = ColumnTransformer(
            [('resume_txt', TfidfVectorizer(), 'Text_Processed'),
             ('Average Incentive [per month] earned in your pervious company ?', MinMaxScaler(), ['Average Incentive [per month] earned in your pervious company ?'])],
            remainder='drop') 

    pipeline = Pipeline([('prep',column_trans),
                         ('clf', clf)])

    return pipeline

In [57]:
# using only resumes
df1 = pd.DataFrame(resume_text_new, columns=['resume_text'])
df1 = pd.concat([df1, train_df['Performance']], axis=1)
df1

,resume_text,Performance
0,ASUHU is a highly skilled and experienced sa...,0
1,Candidate Name: S.Thanga Durai\n Relevant...,1
2,Harun is an experienced finance professional...,1
3, Knowledge of software skills in asset manage...,1
4,This candidate has over one year of experien...,0
...,...,...
740,This candidate has extensive experience in m...,0
741,B. Ravi XXX is an experienced banking profes...,1
742,This candidate has over 7 years of experienc...,1
743,Name: [REDACTED]\n Education:\n - Govt...,1


In [59]:
train_df1, test_df1 = train_test_split(
    df1,
    test_size=0.1
    # stratify = df['Performance'].values
)

In [61]:
X = list(df1['resume_text'])
y = list(df1['Performance'])
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 600)
vectors = tfidf_vectorizer.fit_transform(X)

vectors = vectors.toarray()
vectors

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.06197614],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07821654],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.06841222],
       [0.        , 0.        , 0.        , ..., 0.        , 0.2789151 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.12175852]])

In [62]:
X_train,X_test,y_train,y_test=train_test_split(vectors, y, test_size=0.1, random_state=9491)
# y_train_flat = np.argmax(y_train, axis=1)
# y_test_flat = np.argmax(y_test, axis=1)

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
model = LogisticRegression()

model.fit(X_train, y_train)

predictions = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, predictions))
print("F1-score:", f1_score(y_test, predictions, average='weighted'))

Accuracy: 0.5733333333333334
F1-score: 0.513984962406015


In [64]:
from sklearn.svm import LinearSVC
svc =  LinearSVC()
svc.fit(X_train, y_train)

y_pred_train = svc.predict(X_train)
y_pred_test = svc.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))
print("F1-score:", f1_score(y_test, y_pred_test, average='weighted'))


Training Accuracy score: 0.8985074626865671
Testing Accuracy score: 0.5333333333333333
F1-score: 0.5351196172248803


C:\Users\91939\anaconda3\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [65]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

y_pred_train = gnb.predict(X_train)
y_pred_test = gnb.predict(X_test)
print("F1-score:", f1_score(y_test, y_pred_test, average='weighted'))

F1-score: 0.45996077732216084


In [66]:
from sklearn.ensemble import RandomForestClassifier

crf = RandomForestClassifier(max_depth=5, min_samples_leaf=20, n_estimators=50,
                       n_jobs=-1, random_state=42)

crf.fit(X_train, y_train)
y_pred_train = crf.predict(X_train)
y_pred_test = crf.predict(X_test)
print("F1-score:", f1_score(y_test, y_pred_test, average='weighted'))

F1-score: 0.49951219512195116


In [67]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10,25,30,50,100,200]
}

from sklearn.model_selection import GridSearchCV

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf,
                           param_grid=params,
                           cv = 4,
                           n_jobs=-1, verbose=1, scoring="accuracy")


grid_search.fit(X_train, y_train)
grid_search.best_score_

Fitting 4 folds for each of 180 candidates, totalling 720 fits


0.6029637154262902

In [131]:
rf_best = grid_search.best_estimator_
rf_best

RandomForestClassifier(max_depth=5, min_samples_leaf=20, n_estimators=50,
                       n_jobs=-1, random_state=42)

In [8]:
import pandas as pd
import pandas as pd
import numpy as np
from time import time
import hvplot.pandas
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
data = pd.read_csv('data.csv')

In [9]:
X = list(data['resume_txt'])
y = list(data['Performance'])
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 600)
vectors = tfidf_vectorizer.fit_transform(X)

vectors = vectors.toarray()

In [10]:
from autogluon.core.dataset import TabularDataset
train_data = TabularDataset('data.csv')
train_data.head(30)
train_data.columns

Index(['Performance', 'resume_txt'], dtype='object')

In [13]:
from autogluon.tabular import TabularPredictor # Assuming you're using AutoGluon

predictor = TabularPredictor(label='Performance', eval_metric='f1').fit(train_data,time_limit=100, presets='high_quality' )

No path specified. Models will be saved in: "AutogluonModels\ag-20240411_142804"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 100 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240411_142804/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 25s
AutoGluon will save models to "AutogluonModels\ag-20240411_142804/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.0.